In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np
import time

In [2]:
class MNISTNetwork(Overlay):
    def __init__(self, bitstream_name, batch_size=32):
        super().__init__(bitstream_name)
        self.last_time = 0
        self.inbuf = allocate((batch_size*25,), dtype=np.int32)
        self.outbuf = allocate((batch_size*10,), dtype=np.int32)
        

    def calculate(self, inArr):          
        np.copyto(self.inbuf, inArr)

        self.dma_in.sendchannel.transfer(self.inbuf)
        self.dma_out.recvchannel.transfer(self.outbuf)
        self.dma_in.sendchannel.wait()
        self.dma_out.recvchannel.wait()     

        return self.outbuf
    

In [3]:
batch_size = 512

In [4]:
mnist = np.load("dataset/mnist-original.npy", allow_pickle=True)
X = mnist.item().get("data").T
y = mnist.item().get("label")[0]

model_bit = np.load("dataset/mnist-bit_B{}.npy".format(batch_size), allow_pickle=True).item()
X_bit = model_bit["X"]
y_bit = model_bit["y"]

In [5]:
## for sw
sign = lambda x: float(1) if x>0 else float(-1)
sign = np.vectorize(sign)
adj = lambda x: x*2-1
adj = np.vectorize(adj)
model = np.load("weights/model.npy", allow_pickle=True).item()
fc1w_q = sign(np.array(model['fc1w']))
fc2w_q = sign(np.array(model['fc2w']))
fc3w_q = sign(np.array(model['fc3w']))
 


In [6]:
def feed_foward(X0):
    X0_q = sign(adj(X0))
    
    X1 = np.matmul(X0_q, fc1w_q.T) 
   
    X1_q = sign(X1)
        
    X2 = np.matmul(X1_q, fc2w_q.T)

    X2_q = sign(X2)

    X3 = np.matmul(X2_q, fc3w_q.T) 
    return X3
    
def feed_foward_hw(mnist_hw, X0_bit):
    return mnist_hw.calculate(X0_bit)

In [7]:
mnist_hw = MNISTNetwork("bitstream/mnist_B{}.bit".format(batch_size), batch_size)

prediction = []
score = 0
i = 0
assert(len(X_bit) == len(y_bit))

t1 = time.time()
for i in range(len(X_bit)):
    xs, ys = X_bit[i], y_bit[i]    
    outputs = feed_foward_hw(mnist_hw, xs)
    for i in range(batch_size):
        prediction.append(np.argmax(outputs[i*10:(i+1)*10])== ys[i])
score = np.mean(prediction) * 100
print(score)
t2 = time.time()
print(t2-t1)

89.05101102941177
32.58093333244324


In [ ]:
prediction = []
score = 0
i = 0
t1 = time.time()


for idx in range(len(X)//batch_size):
    xs = X[batch_size * idx:batch_size * idx + batch_size]
    ys = y[batch_size * idx:batch_size * idx + batch_size]
    outputs = feed_foward(xs)
    for output, yk in zip(outputs, ys):
        prediction.append(np.argmax(output) == (yk))
    i += 1
#     print("{}th iter".format(i))
    
score = np.mean(prediction) * 100
print(score)
t2 = time.time()
print(t2-t1)